In [3]:
print("all ok")

all ok


In [4]:
import os
from dotenv import load_dotenv

In [5]:
os.environ["HUGGING_FACE_KEY"]=os.getenv("HUGGING_FACE_KEY")

In [6]:
### Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-l6-v2")

In [7]:
### Index
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
index=faiss.IndexFlatL2(384)

In [8]:
### Vectore store
vector_store= FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [9]:
doucments=["You can reset your password by clicking 'Forgot Password' on the login page.",
    "Our return policy allows returns within 30 days of purchase with a receipt.",
    "Shipping typically takes 3-5 business days for domestic orders."]

In [10]:
vector_store.add_texts(doucments)

['d94eb7c8-3324-4f51-ae91-a766ce1d90a3',
 '82f58c95-f5cb-482a-b820-e8f5f9a8180c',
 'ce6edd83-e436-4c75-b988-7fffa13acad3']

In [11]:
### Retriever
retriever=vector_store.as_retriever(search_kwargs={"k":3})

In [12]:
### LLM
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [13]:
### output parser
from langchain_core.output_parsers import StrOutputParser
outputParser =StrOutputParser()

In [14]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [15]:
### toxicity validation
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

toxic_model_name = "toxic_bert_finetuned"
tokenizer = AutoTokenizer.from_pretrained(toxic_model_name)
toxic_model = AutoModelForSequenceClassification.from_pretrained(toxic_model_name)

def is_toxic(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = toxic_model(**inputs)
        scores = torch.sigmoid(outputs.logits)[0].numpy()
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    result = dict(zip(labels, scores))
    toxic_score = result["toxic"]
    return toxic_score > 0.5  # threshold on toxicity scores

In [16]:
#### PROMPT template and RAG chain
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    print("-> RAG Call question->"+question)
    
    prompt=PromptTemplate(
        template="""
        You are a helpful customer support assistant. Use only the provided context to answer. 
        If you don't know the answer, say: 'Please connect with a support agent.'

        Context:
        {context}

        Question:
        {question}
        """,

        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    user_question = state["messages"][-1]
    if user_question.lower() in ["quit", "exit"]:
        return print("Bye")
    if is_toxic(user_question):
        print("AI Assistant: I'm here to help with respectful communication. Please rephrase your question politely.")
    else:
        result = rag_chain.invoke(question)
        return {"messages": [result]}

In [17]:
state={"messages":["i forgot my password.please help"]}

In [18]:
function_1(state)

-> RAG Call question->i forgot my password.please help
AI Assistant: I'm here to help with respectful communication. Please rephrase your question politely.


In [19]:
state={"messages":["you are so useless."]}

In [20]:
function_1(state)

-> RAG Call question->you are so useless.
AI Assistant: I'm here to help with respectful communication. Please rephrase your question politely.
